In [ ]:
%matplotlib inline

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from remapping import mom_remapping
import gsw
import hycom
from importlib import reload

In [ ]:
temp_url = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/temperature/netcdf/decav/1.00/woa13_decav_t00_01v2.nc'
salt_url = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/salinity/netcdf/decav/1.00/woa13_decav_s00_01v2.nc'

temp_url_025 = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/temperature/netcdf/A5B2/0.25/woa13_A5B2_t00_04v2.nc'
salt_url_025 = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/salinity/netcdf/A5B2/0.25/woa13_A5B2_s00_04v2.nc'

In [ ]:
temp = Dataset(temp_url, 'r')
salt = Dataset(salt_url, 'r')

In [ ]:
lat = temp.variables['lat'][:]
lon = temp.variables['lon'][:]
dep = temp.variables['depth'][:]

# Remapping

We define a function that generates a remapping function.

In [ ]:
remap_cs = mom_remapping.Remapping_Cs()
remap_cs.remapping_scheme = 4 # PQM_IH4IH3
remap_cs.degree = 4

def gen_remap(gr_th, sa_sect, ct_sect):
    def remap(h):
        sa_remap = np.empty_like(h)
        ct_remap = np.empty_like(h)

        # remap by columns
        for i in range(h.shape[1]):
            # we need to make sure we deal with unmasking here,
            # otherwise we'll get the fill values for thickness
            # and salt/temp, which would be just a little weird
            sa_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                           sa_sect[:,i].compressed(),
                                                           h[:,i], remap_cs)
            ct_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                           ct_sect[:,i].compressed(),
                                                           h[:,i], remap_cs)

        return sa_remap, ct_remap
    
    return remap

# Climatology Data

Define a function to return a remapping function for a transect.

In [ ]:
def transect(lon_i, lat_i, ns=True):
    # get temp and salt data for this transect
    lat_sect = lat[lat_i]
    lon_sect = lon[lon_i]
    t_sect = temp.variables['t_an'][0,:,lat_i,lon_i].squeeze()
    s_sect = salt.variables['s_an'][0,:,lat_i,lon_i].squeeze()

    # empty columns are entirely masked - something crashes otherwise
    empty = np.sum(~t_sect.mask, axis=0) == 0
    if ns:
        lat_sect = lat_sect[~empty]
    else:
        lon_sect = lon_sect[~empty]
    t_sect = t_sect[:,~empty]
    s_sect = s_sect[:,~empty]

    # calculate SA and CT data
    sa_sect = np.empty_like(s_sect)
    ct_sect = np.empty_like(t_sect)

    for i in range(s_sect.shape[1]):
        # calculate columnwise
        if ns:
            sa_sect[:,i] = gsw.SA_from_SP(s_sect[:,i], dep, lon_sect, lat_sect[i])
        else:
            sa_sect[:,i] = gsw.SA_from_SP(s_sect[:,i], dep, lon_sect[i], lat_sect)
        ct_sect[:,i] = gsw.CT_from_t(sa_sect[:,i], t_sect[:,i], dep)

    # interpolate interfacial measurements onto layers
    sa_lay = (sa_sect[1:,:] + sa_sect[:-1,:]) / 2
    ct_lay = (ct_sect[1:,:] + ct_sect[:-1,:]) / 2

    # depths of all interfaces on which observations are present
    gr_int = np.ma.array(np.tile(dep.reshape(-1, 1), (1, sa_sect.shape[1])),
                         mask=sa_sect.mask)
    # thicknesses of all layers between interfaces
    gr_th  = np.diff(gr_int, axis=0)
    # bottom interface at each column
    topo = gr_int.max(axis=0)
    
    return lon_sect, lat_sect, gen_remap(gr_th, sa_sect, ct_sect), topo

# Replicating the MOM6 HyCOM grid

As a point of comparison, we'll implement MOM6's 75-level HyCOM grid on our climatology section, which will hopefully let us see what's going wrong with it, and whether any of our ideas can help.

Above, we used the 50-level target densities from the global_ALE experiment. Now we'll use the 75-level targets from OM4_025. The depth coordinates are generated by the string `FNC1:2,4000,4.5,.01`.

In [ ]:
s2_75 = [1010, 1014.3034, 1017.8088, 1020.843, 1023.5566, 1025.813, 
    1027.0275, 1027.9114, 1028.6422, 1029.2795, 1029.852, 1030.3762, 
    1030.8626, 1031.3183, 1031.7486, 1032.1572, 1032.5471, 1032.9207, 
    1033.2798, 1033.6261, 1033.9608, 1034.2519, 1034.4817, 1034.6774, 
    1034.8508, 1035.0082, 1035.1533, 1035.2886, 1035.4159, 1035.5364, 
    1035.6511, 1035.7608, 1035.8661, 1035.9675, 1036.0645, 1036.1554, 
    1036.2411, 1036.3223, 1036.3998, 1036.4739, 1036.5451, 1036.6137, 
    1036.68, 1036.7441, 1036.8062, 1036.8526, 1036.8874, 1036.9164, 
    1036.9418, 1036.9647, 1036.9857, 1037.0052, 1037.0236, 1037.0409, 
    1037.0574, 1037.0738, 1037.0902, 1037.1066, 1037.123, 1037.1394, 
    1037.1558, 1037.1722, 1037.1887, 1037.206, 1037.2241, 1037.2435, 
    1037.2642, 1037.2866, 1037.3112, 1037.3389, 1037.3713, 1037.4118, 
    1037.475, 1037.6332, 1037.8104, 1038]

In [ ]:
def dz_f1(n, dz_min, total, power, precision):
    dz = np.empty(n)
    
    # initial profile
    for i in range(n):
        dz[i] = (i / (n - 1)) ** power
    
    # rescale to total depth and round to precision
    dz[:] = (total - n*dz_min) * (dz[:] / np.sum(dz))
    dz[:] = np.around(dz[:], decimals=precision)
    
    # adjust bottom
    dz[-1] += total - np.sum(dz[:] + dz_min)
    dz[-1] = np.around(dz[-1], decimals=precision)
    
    dz[:] += dz_min
    
    return dz

In [ ]:
dz_75 = dz_f1(75, 2, 4000, 4.5, 2)
max_int_depth = np.insert(dz_f1(75, 5, 8000, 1, 2).cumsum(), 0, 0)
max_lay_thick = dz_f1(75, 400, 31000, 0.1, 2)

Calculate nominal grid spacings and adjust to local topography.

In [ ]:
def gen_z_75_sect(topo):
    # convert to interface positions
    z_75 = np.insert(dz_75.cumsum(), 0, 0)[:,np.newaxis]
    # tile to the full size of the transect
    z_75_full = np.tile(z_75, (1, topo.size))
    # clip below topography
    np.putmask(z_75_full, z_75_full > topo, topo)
    
    return z_75_full

Calculate maximum interface depths and maximum layer thicknesses, also from the `dz_f1()` generating function.

In [ ]:
def run_hycom(topo, remap, **kwargs):
    # calculate interface positions for this transect
    z_75 = gen_z_75_sect(topo)
    # convert to layer thicknesses
    h = np.diff(z_75, axis=0)
    # remap state
    sa, ct = remap(h)
    
    # use hycom algorithm to give:
    # - z_iso:   interpolated isopycnal positions
    # - z_bnd:   interface positions according to original hycom algorithm
    # - z_bnd_a: interface positions with adjusted algorithm
    z_iso, z_bnd, z_bnd_a = \
        hycom.hycom(h, sa, ct, s2_75, dz_75, max_int_depth, max_lay_thick, **kwargs)
        
    # np.diff(z, axis=0)
    return z_iso, z_bnd, z_bnd_a

# Denmark Strait

First we have a full section through the Atlantic from the Denmark Strait down to the Antarctic margin

In [ ]:
# all latitudes up to Greenland
lat_i = slice(None, 170)
lon_i = np.abs(lon - (-25.5)).argmin()

lon_den, lat_den, remap_den, topo_den = transect(lon_i, lat_i)

zi, zb, za = run_hycom(topo_den, remap_den, s_topo=True, detangle=True)
sa_iso, _ = remap_den(np.diff(zi, axis=0))
sa_bnd, _ = remap_den(np.diff(zb, axis=0))
sa_adj, _ = remap_den(np.diff(za, axis=0))

In [ ]:
plt.figure(figsize=(10,6))

plt.pcolormesh(lat_den, zi, sa_iso)
plt.plot(lat_den, zi.T, 'k', linewidth=0.5)
plt.gca().invert_yaxis()
plt.colorbar()
plt.title('target isopycnals');

In [ ]:
plt.figure(figsize=(10,8))

l = 55
b = 300
xh = lat_den

ax = plt.subplot(121)
plt.pcolormesh(xh, zb, sa_bnd)
plt.plot(xh, zb.T, 'k', linewidth=0.5)
ax.invert_yaxis()
plt.ylabel('depth')
plt.title('regular transition')

plt.xlim(left=l)
plt.ylim(bottom=b)

ax = plt.subplot(122)
plt.pcolormesh(xh, za, sa_adj)
plt.plot(xh, za.T, 'k', linewidth=0.5)
ax.invert_yaxis()
ax.set_yticks([])
plt.title('adjusted transition')

plt.xlim(left=l)
plt.ylim(bottom=b)

plt.tight_layout()

# Gibraltar Strait

In [ ]:
lat_i = np.abs(lat - 36).argmin()
lon_i = slice(np.abs(lon - (-20)).argmin(),
              np.abs(lon - (-5.5)).argmin())

lon_gib, lat_gib, remap_gib, topo_gib = transect(lon_i, lat_i, ns=False)

zi, zb, za = run_hycom(topo_gib, remap_gib, s_topo=True, detangle=True)
sa_iso, _ = remap_gib(np.diff(zi, axis=0))
sa_bnd, _ = remap_gib(np.diff(zb, axis=0))
sa_adj, _ = remap_gib(np.diff(za, axis=0))

In [ ]:
plt.figure(figsize=(10,6))

plt.pcolormesh(lon_gib, zi, sa_iso)
plt.plot(lon_gib, zi.T, 'k', linewidth=0.5)
plt.gca().invert_yaxis()
plt.colorbar()
plt.title('target isopycnals');

In [ ]:
plt.figure(figsize=(10,8))

l = None
b = None
xh = lon_gib

ax = plt.subplot(121)
plt.pcolormesh(xh, zb, sa_bnd)
plt.plot(xh, zb.T, 'k', linewidth=0.5)
ax.invert_yaxis()
plt.ylabel('depth')
plt.title('regular transition')

plt.xlim(left=l)
plt.ylim(bottom=b)

ax = plt.subplot(122)
plt.pcolormesh(xh, za, sa_adj)
plt.plot(xh, za.T, 'k', linewidth=0.5)
ax.invert_yaxis()
ax.set_yticks([])
plt.title('adjusted transition')

plt.xlim(left=l)
plt.ylim(bottom=b)

plt.tight_layout()

# Gulf Stream

In [ ]:
lat_i = np.abs(lat - 30).argmin()
lon_i = slice(np.abs(lon - (-81)).argmin(),
              np.abs(lon - (-60)).argmin())

lon_gul, lat_gul, remap_gul, topo_gul = transect(lon_i, lat_i, ns=False)

zi, zb, za = run_hycom(topo_gul, remap_gul, s_topo=True, detangle=True)
sa_iso, _ = remap_gul(np.diff(zi, axis=0))
sa_bnd, _ = remap_gul(np.diff(zb, axis=0))
sa_adj, _ = remap_gul(np.diff(za, axis=0))

In [ ]:
plt.figure(figsize=(10,6))

plt.pcolormesh(lon_gul, zi, sa_iso)
plt.plot(lon_gul, zi.T, 'k', linewidth=0.5)
plt.gca().invert_yaxis()
plt.colorbar()
plt.title('target isopycnals');

In [ ]:
plt.figure(figsize=(10,8))

l = None
b = None
xh = lon_gul

ax = plt.subplot(121)
plt.pcolormesh(xh, zb, sa_bnd)
plt.plot(xh, zb.T, 'k', linewidth=0.5)
ax.invert_yaxis()
plt.ylabel('depth')
plt.title('regular transition')

plt.xlim(left=l)
plt.ylim(bottom=b)

ax = plt.subplot(122)
plt.pcolormesh(xh, za, sa_adj)
plt.plot(xh, za.T, 'k', linewidth=0.5)
ax.invert_yaxis()
ax.set_yticks([])
plt.title('adjusted transition')

plt.xlim(left=l)
plt.ylim(bottom=b)

plt.tight_layout()